#Transfer Learning with TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
def find_min_max_dimensions(directory):
    max_height = 0
    max_width = 0
    min_height = float('inf')
    min_width = float('inf')

    # Loop through all files in the directory and its subdirectories
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):  # Check common image extensions
                img_path = os.path.join(subdir, file)
                height, width = get_image_dimensions(img_path)

                # Update max dimensions
                max_height = max(max_height, height)
                max_width = max(max_width, width)

                # Update min dimensions
                min_height = min(min_height, height)
                min_width = min(min_width, width)

    return min_height, min_width, max_height, max_width

In [ ]:
root_directory = 'drive/MyDrive/scans'
min_height, min_width, max_height, max_width = find_min_max_dimensions(root_directory)

print(f"Min Height: {min_height}, Min Width: {min_width}")
print(f"Max Height: {max_height}, Max Width: {max_width}")

Min Height: 310, Min Width: 190
Max Height: 719, Max Width: 1048


In [ ]:
def resize_and_pad_image(img_path, target_size=224):
    # Load the image
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)

    # Calculate the aspect ratio and resize
    original_height, original_width, _ = img.shape
    aspect_ratio = original_width / original_height
    if original_width < original_height:
        new_width = target_size * aspect_ratio
        new_height = target_size
    else:
        new_height = target_size / aspect_ratio
        new_width = target_size
    img = tf.image.resize(img, [int(new_height), int(new_width)])

    # Pad to target size
    img = tf.image.resize_with_crop_or_pad(img, target_size, target_size)

    # Convert image to uint8 and encode to JPEG for saving
    img = tf.image.convert_image_dtype(img, dtype=tf.uint8)
    img_encoded = tf.image.encode_jpeg(img)
    return img_encoded

def process_and_save_images(source_directory, destination_directory, target_size=224):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    for subdir, _, files in os.walk(source_directory):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                # Construct full file path
                source_path = os.path.join(subdir, file)

                # Construct destination path
                relative_path = os.path.relpath(subdir, source_directory)
                dest_subdir = os.path.join(destination_directory, relative_path)
                dest_path = os.path.join(dest_subdir, file)

                # Ensure destination directory exists
                if not os.path.exists(dest_subdir):
                    os.makedirs(dest_subdir)

                # Process and save the image
                processed_img = resize_and_pad_image(source_path, target_size)
                tf.io.write_file(dest_path, processed_img)

                print(f"Processed and saved: {dest_path}")

In [ ]:
process_and_save_images("drive/MyDrive/scans", "drive/MyDrive/new_scans")

Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_93_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_92_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_48_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_77_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_49_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_124_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_125_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_108_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_4_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_26_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_18_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_19_mask.png
Processed and saved: drive/MyDrive/new_scans/normal/masks/normal_27_mask.p